## Loading Data

In [1]:
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.svm import SVR
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.kernel_ridge import KernelRidge


from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.neighbors import KNeighborsRegressor
from sklearn import linear_model



/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
dfFinal = pd.read_csv('News_Final.csv')
dfFb_Ec = pd.read_csv('Facebook_Economy.csv')
dfFb_Ms = pd.read_csv('Facebook_Microsoft.csv')
dfFb_Ob = pd.read_csv('Facebook_Obama.csv')
dfFb_Pa = pd.read_csv('Facebook_Palestine.csv')
dfGo_Pa = pd.read_csv('GooglePlus_Palestine.csv')
dfGo_Ob = pd.read_csv('GooglePlus_Obama.csv')
dfGo_Ms = pd.read_csv('GooglePlus_Microsoft.csv')
dfGo_Ec = pd.read_csv('GooglePlus_Economy.csv')
dfLd_Pa = pd.read_csv('LinkedIn_Palestine.csv')
dfLd_Ob = pd.read_csv('LinkedIn_Obama.csv')
dfLd_Ms = pd.read_csv('LinkedIn_Microsoft.csv')
dfLd_Ec = pd.read_csv('LinkedIn_Economy.csv')

In [3]:
#Edit original Data frame to assign group rank
def assignRank(row,rankDictionary):
    if row.IDLink in rankDictionary:
        return rankDictionary[row.IDLink]
    else:
        return 1

In [4]:
def rank_model_preprocess(df):
    #removing time from publish date
    df['PublishDate'] = df['PublishDate'].astype('datetime64[ns]')
    df['DateOnly'] = df['PublishDate'].dt.date

    #count number of days since Jan 1 1970. These columns would be used to form groups of 3 days
    df['DaysSince1'] = (pd.to_datetime(df['DateOnly']) - pd.datetime(1970,1,1)).dt.days
    df['DaysSince1'] = df['DaysSince1'].subtract(df['DaysSince1'].min()).add(1)
    df['DaysSince2'] = df['DaysSince1'] + 1
    df['DaysSince3'] = df['DaysSince1'] + 2

    print "Calculating rank..."
    # Calculate rank of the news item within the 3 day window group

    loop_start = df['DaysSince1'].min()
    loop_end = df['DaysSince1'].max()
    print loop_start
    print loop_end
    rankDictionary = {}
    for x in range(loop_start, loop_end):
        df_temp = df[(df.DaysSince1 == x) | (df.DaysSince2 == x) | (df.DaysSince3 == x)]
        df_temp.sort_values(by='Facebook')# Made it ascending so that we rank them in reverse order #, ascending=False)
        df_temp['GroupRanking'] = df_temp['Facebook'].rank(ascending=False)
        for index, row in df_temp.iterrows():
            if row['DaysSince1'] == x:
                rankDictionary[row['IDLink']] = row['GroupRanking']
    df['groupRank'] = df.apply (lambda row: assignRank (row,rankDictionary),axis=1)
    #df['Facebook'] = df['Facebook'].apply(lambda x: np.log(x + 1))
    #Split DataFrame into X and Y
    #df_X_Without_Rank = pd.get_dummies(df[['Source', 'Topic', 'SentimentTitle', 'SentimentHeadline']])
    df_X = pd.get_dummies(df[['Source', 'Topic', 'SentimentTitle', 'SentimentHeadline', 'groupRank']])
    df_Y = df[['Facebook']]
    #xtrain, xtest, ytrain, ytest = train_test_split(df_X, df_Y, test_size=0.25)
    return df_X,df_Y

In [6]:
df_model = dfFinal[(dfFinal.Facebook != -1) & (dfFinal.GooglePlus != -1) & (dfFinal.LinkedIn != -1)]
df_X,df_Y = rank_model_preprocess(df_model)
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)
trunacted_X= svd.fit_transform(df_X)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

Calculating rank...
1
243


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
 df_Y['Facebook'] = df_Y['Facebook'].apply(lambda x: np.log(x + 1))

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:

def dr_model(trunacted_X,df_Y):
     
    xtrain, xtest, ytrain, ytest = train_test_split(trunacted_X, df_Y, test_size=0.25)
    return df_model, xtrain, ytrain,xtest, ytest


In [10]:

models= [dr_model(trunacted_X,df_Y)]
model_names = ["DR Model" ]

In [14]:


ModelEvaluationResults = {}




results = {}

for (m,models_) in enumerate(models):
    df,train_X,train_Y,test_X,test_Y = models_   
    print "Evalutaion of Model:  "+str(model_names[m])
    
    clfs = [ 
        
    SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.2, gamma='auto',kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
    #,linear_model.BayesianRidge(),
    ,linear_model.LassoLars()
    #KernelRidge(alpha=1.0, coef0=1, degree=3, gamma=None, kernel='linear',kernel_params=None)]

    #clf_names = ['Logistic Regression','KNeighborsRegressor','SVR','KernelRidge']
    #clf_names = ['SVR','BayesianRidge','LassoLars'
                ]
    clf_names = ['SVR','LassoLars'
                ]

    
    ClassifierEvaluationResults = {}
    
    for (i, clf_) in enumerate(clfs):
        print "********************************************************************"
        print "Training validations evaluations for : "+str(clf_names[i])
        scores = cross_val_score(clf_, train_X, train_Y, cv=5)
        predicted_ratings = cross_val_predict(clf_, test_X,test_Y, cv=5)
        print scores
        train_mean_score=scores.mean()
        print "Testing validations evaluations for : "+str(clf_names[i])


        r2 = r2_score(test_Y, predicted_ratings, multioutput='uniform_average')
        mae = median_absolute_error(test_Y, predicted_ratings)
        msle =mean_squared_log_error(test_Y, predicted_ratings) 
        mse = mean_squared_error(test_Y, predicted_ratings)
        mae = mean_absolute_error(test_Y, predicted_ratings)
        evs = explained_variance_score(test_Y, predicted_ratings)  
        
        data ={'training score':train_mean_score,
            'r2 score':r2,
            'median absolute error':mae,
            'mean squared log error':msle,
            'mean sqaured error':mse,
            'mean absolute error':mae,
            'explained variance error':evs}
   
        
        ClassifierEvaluationResults[clf_names[i]] = data
        print "********************************************************************"
        print data
        print "********************************************************************"
        
    print "*******************Evaluation Done*************************************************"
    ModelEvaluationResults[model_names[m]]=ClassifierEvaluationResults
    print ClassifierEvaluationResults
    print "********************************************************************"

Evalutaion of Model:  DR Model
********************************************************************
Training validations evaluations for : SVR
[0.87121535 0.87775333 0.88069807 0.88079504 0.87460336]
Testing validations evaluations for : SVR
********************************************************************
{'r2 score': 0.8752970542647409, 'mean absolute error': 0.5159980409259136, 'mean sqaured error': 0.5219218128341803, 'training score': 0.8770130282946786, 'median absolute error': 0.5159980409259136, 'explained variance error': 0.876018578338955, 'mean squared log error': 0.08394990491534288}
********************************************************************
********************************************************************
Training validations evaluations for : LassoLars
[-4.90746485e-05 -8.76276147e-05 -5.67826072e-05 -1.26350496e-04
 -5.58300496e-06]
Testing validations evaluations for : LassoLars
********************************************************************
{'r2 sc